# 1 Initialization

In order to execute the individual solver runs,
we are going to employ the mini batch processor,
for running the calculations on the local machine.
We also have to initialize the workflow management system and create 
a database.

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [ ]:
/// BoSSSexeSilent BoSSScmdSilent
using NUnit.Framework;

Error: (2,7): error CS0246: Der Typ- oder Namespacename "NUnit" wurde nicht gefunden (möglicherweise fehlt eine using-Direktive oder ein Assemblyverweis).

In [ ]:
BoSSSshell.WorkflowMgm.Init("ConvStudyTutorial");

Project name is set to 'ConvStudyTutorial'.


In [ ]:
var db = CreateTempDatabase();

Creating database 'C:\Users\flori\AppData\Local\Temp\1035387140'.


The following (deactivated) line would delete all Sessions (i.e. solver 
runs) which correspond to this project from the database.
Hence, on every execution of the worksheet, all simulations would be 
re-done. 

Normally, without the following line, existing simulations from 
the database will be used; therefore, it is save to close and open
the worksheet.

This is handy e.g. when simulations are running on a cluster for a long 
time, and we usually don't want to re-submit the calculation 
every time we execute the worksheet.

```csharp
BoSSSshell.WorkflowMgm.Sessions.ForEach(si => si.Delete(true));
```

For sake of simplicity, we employ the Poisson solver
**ipPoisson** which is just a benchmarking application, but sufficient 
for the purpose of this tutorial.

In [ ]:
using BoSSS.Application.SipPoisson;

Error: (1,7): error CS0246: Der Typ- oder Namespacename "BoSSS" wurde nicht gefunden (möglicherweise fehlt eine using-Direktive oder ein Assemblyverweis).

We also instantiate a client for the **MiniBatchProcessor**:

In [ ]:
var myBatch = ExecutionQueues[0];

In [ ]:
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ C:\ == , D:\ == ]"


# 2 Mesh Creation

We create multiple grids using resolutions of $2 \times 2$, $4 \times 4$ to $32 \times 32$
cells:

In [ ]:
int[] resolutions = new int[] { 2, 4, 8, 16, 32};

In [ ]:
var grids = new GridCommons[resolutions.Length];
for(int iRes = 0; iRes < resolutions.Length; iRes++) {
    // create nodes:
    var Nodes = GenericBlas.Linspace(-Math.PI*0.5, Math.PI*0.5, 
        resolutions[iRes] + 1); // note: number of nodes = number of cells + 1!
 
    // create grid:
    GridCommons grd_i = Grid2D.Cartesian2DGrid(Nodes, Nodes);
 
    // define all boundaries as Dirichlet:
    grd_i.EdgeTagNames.Add(1, BoundaryType.Dirichlet.ToString());
    grd_i.DefineEdgeTags(delegate (double[] X) {
        byte ret = 1;
        return ret;
    });
 
    // save grid in database
    db.SaveGrid(ref grd_i);
 
    // remenber reference to grid:
    grids[iRes] = grd_i;
}

Grid Edge Tags changed.
Grid Edge Tags changed.
Grid Edge Tags changed.
Grid Edge Tags changed.
Grid Edge Tags changed.


# 3 Setup and execution of solver runs

First, we implement the exact expressions for the right-hand-side $$ f(x,y)= -2\cos(x)\cos(y)$$
and the exact solution.
$$ u_{sol} (x,y)=\cos(x) \cos(y) $$

The exact solution will be used to compute the error of the simulation.
Normally, the exact solution is not known; in those cases, we need to 
compute the experimental convergence against the solution on the finest
grid.

In [ ]:
string formula_code = 
"static class Expressions { " +
"    public static double RHS(double[] X) { " +
"        double x = X[0]; " +
"        double y = X[1]; " +
"        return -2.0*Math.Cos(x)*Math.Cos(y); " +
"    } " +
"    public static double Tex(double[] X) { " +
"        double x = X[0]; " +
"        double y = X[1]; " +
"        return Math.Cos(x)*Math.Cos(y); " +
"    } " +
"}";

In [ ]:
var RHSfunc = new Formula("Expressions.RHS", false, formula_code);

In [ ]:
var TexFunc = new Formula("Expressions.Tex", false, formula_code);

We compute 4 different polynomial orders:

In [ ]:
int[] Degrees = new int[] {1, 2, 3, 4};

Setup of all runs...

In [ ]:
var Runs = new List<SipControl>();
Runs.Clear(); // start with an empty run list
foreach(int pDeg in Degrees) { // loop over polynomial degrees
foreach(var grd in grids) { //    loop over all grids
 
    // create object and remember in list:
    SipControl C = new SipControl();   
    Runs.Add(C);
 
    // set polynomial degree and grid:
    C.SetDGdegree(pDeg);
    C.SetGrid(grd);
 
    // specify RHS and exact solution (these are realized as initial values
    // in the \code{ipPoisson} solver:
    C.AddInitialValue("RHS", RHSfunc);
    C.AddInitialValue("Tex", TexFunc);
 
    // specify boundary condition:
    C.AddBoundaryValue(BoundaryType.Dirichlet.ToString()); // for homogeneous 
    //   boundary conditions, we don not need a value, since the default value
    //   zero is sufficient.
 
    // the exact solution is a speciality of the SIP Poisson benchmark solver;
    // in order to evaluate the exact solution, we have to set the following 
    // boolean:
    C.ExactSolution_provided = true;
 
}
}

...and activate them:

In [ ]:
foreach(var C in Runs)
   C.RunBatch(myBatch);

The following line ensures that all jobs are complete before 
post-processing analysis is started, although, there is a one-hour (3600-seconds )
time-out.

In [ ]:

BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(3600*4);

Loading session 030ce030-4bfb-40aa-b7e9-030d528f5253 failed with message 'Could not find file 'C:\Users\flori\AppData\Local\Temp\1035387140\sessions\030ce030-4bfb-40aa-b7e9-030d528f5253\Session.info'.'
Loading session 1564b61c-f1d2-43b9-b2ff-be986d182f7a failed with message 'Could not find file 'C:\Users\flori\AppData\Local\Temp\1035387140\sessions\1564b61c-f1d2-43b9-b2ff-be986d182f7a\Session.info'.'
Loading session 2d6a664f-2817-476f-bfab-e7035862f680 failed with message 'Could not find file 'C:\Users\flori\AppData\Local\Temp\1035387140\sessions\2d6a664f-2817-476f-bfab-e7035862f680\Session.info'.'
Loading session 3d6f2c96-980c-4983-adbc-48335adac62d failed with message 'Could not find file 'C:\Users\flori\AppData\Local\Temp\1035387140\sessions\3d6f2c96-980c-4983-adbc-48335adac62d\Session.info'.'
Loading session 4524f207-6508-43bf-b65e-db6a66a86fe2 failed with message 'Could not find file 'C:\Users\flori\AppData\Local\Temp\1035387140\sessions\4524f207-6508-43bf-b65e-db6a66a86fe2\Sessio

Note that, in a larger production run study, where jobs may run days or
weeks, blocking the worksheet is not really usefull.
Instead, one might split process into two workseets 
(eactly at this line here), one for set-up and
job sumbission and another one for the analysis.

In [ ]:

BoSSSshell.WorkflowMgm.AllJobs

Error: (1,1): error CS0103: Der Name "BoSSSshell" ist im aktuellen Kontext nicht vorhanden.

We can take a closer inspection of anything that failed (should not be,
anyway).

In [ ]:

foreach(var job in BoSSSshell.WorkflowMgm.AllJobs.Values) {
    if(job.Status != JobStatus.FinishedSuccessful) {
        Console.WriteLine("###############################################");
        Console.WriteLine($"Job {job}");
        Console.WriteLine("###############################################");
        Console.WriteLine(job.Stdout);   
        Console.WriteLine("===============================================");
        Console.WriteLine(job.Stderr);
        Console.WriteLine("###############################################");
    }    
}


# 4 Convergence against exact solution

As already noted, the computation of the $L^2$ error against the 
exact solution is handled specially in the **ipPoisson** solver.
However, the following tutorial can serve as a general template of how to
extract data from the session table and visualize it.

We aquire a copy of the session table, and from all the columns in there...


In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;

In [ ]:
Tab.GetColumnNames().Take(7)  // Take(7) is just to shorten the output. There are a total of 86 ColumnNames.

index,value
0,ProjectName
1,SessionName
2,DGdegree:T
3,DGdegree:Tex
4,Bndtype:Dirichlet
5,NoOfMultigridLevels
6,TimesteppingMode


...we extract those which sound interesting:

In [ ]:
Tab = Tab.ExtractColumns(
    //"SessionName", 
    "DGdegree:T", "Grid:NoOfCells", "Grid:hMin", "DOFs", 
    //"ExactSolution_provided", 
    "SolL2err");

In [ ]:
Tab.Print();

    DGdegree:T Grid:NoOfCells Grid:hMin           DOFs  SolL2err               
0:  4          1024           0.0981747704246807  15360 4.189135477771225E-09  
1:  3          1024           0.0981747704246807  10240 3.3925701613103797E-07 
2:  4          256            0.19634954084936185 3840  1.3463890974512488E-07 
3:  3          256            0.19634954084936185 2560  5.584910636792265E-06  
4:  4          64             0.39269908169872414 960   4.366082710599254E-06  
5:  3          64             0.39269908169872414 640   9.849416925455182E-05  
6:  4          16             0.7853981633974483  240   0.00014480970262714088 
7:  4          4              1.5707963267948966  60    0.004890481980291057   
8:  3          16             0.7853981633974483  160   0.001914897579083584   
9:  3          4              1.5707963267948966  40    0.03377668103207887    
10: 2          1024           0.0981747704246807  6144  2.568520840537063E-05  
11: 2          256            0.19634954


Note: the session table can also be exported, e.g. to Excel or 
Libre/Open Office Calc, by using the **ToCSVFile** function.

The columns of the session table 
can be easily converted to a plot: the $x$-axis is determined
by the cell width, the $y$-axis is determined by the $L^2$ error.
Furthermore, we want to *group* our plots according
to the DG degree, i.e. have one line for each polynomial degree;

In [ ]:
var ErrorPlot = Tab.ToPlot("Grid:hMin", "SolL2err", // column for x- and y
                           "DGdegree:T"); // column for group

We set logarithmic axes:

In [ ]:

ErrorPlot.LogX = true;
ErrorPlot.LogY = true;

In [ ]:
ErrorPlot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -9 $ 
 
 
 
 
 $10 -8 $ 
 
 
 
 
 $10 -7 $ 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:T4 
 
 
 DGdegree:T4 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:T3 
 
 
 DGdegree:T3 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:T2 
 
 
 DGdegree:T2 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:T1 
 
 
 DGdegree:T1

Of course, we can adjust the plot styles:

In [ ]:
ErrorPlot.dataGroups[0].Format.PointType = PointTypes.Diamond;
ErrorPlot.dataGroups[1].Format.PointType = PointTypes.Box;
ErrorPlot.dataGroups[2].Format.PointType = PointTypes.LowerTriangle;
ErrorPlot.dataGroups[3].Format.PointType = PointTypes.Asterisk;

In [ ]:
foreach(var grp in ErrorPlot.dataGroups) {
    grp.Format.PointSize = 0.8;
    grp.Format.DashType  = DashTypes.Dotted;
    grp.Format.LineWidth = 2;
}

In [ ]:
ErrorPlot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -9 $ 
 
 
 
 
 $10 -8 $ 
 
 
 
 
 $10 -7 $ 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:T4 
 
 
 
 
 DGdegree:T4 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:T3 
 
 
 DGdegree:T3 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:T2 
 
 
 DGdegree:T2 
 
 
 
 
 
 
 
 
 
 
 
 DGdegree:T1 
 
 
 DGdegree:T1

And we can compute the convergence order:

In [ ]:
ErrorPlot.Regression()

Error: (1,1): error CS0103: Der Name "ErrorPlot" ist im aktuellen Kontext nicht vorhanden.

In [ ]:
/// BoSSScmdSilent
var reg1     = ErrorPlot.Regression();
double conv1 = reg1.Single(kv => kv.Key.Contains("T1")).Value;
double conv2 = reg1.Single(kv => kv.Key.Contains("T2")).Value;
double conv3 = reg1.Single(kv => kv.Key.Contains("T3")).Value;
double conv4 = reg1.Single(kv => kv.Key.Contains("T4")).Value;
Assert.IsTrue(Math.Abs(conv1 - (+2)) < 0.7, "experimental convergence failed on k = 1");
Assert.IsTrue(Math.Abs(conv2 - (+3)) < 0.7, "experimental convergence failed on k = 2");
Assert.IsTrue(Math.Abs(conv3 - (+4)) < 0.7, "experimental convergence failed on k = 3");
Assert.IsTrue(Math.Abs(conv4 - (+5)) < 0.7, "experimental convergence failed on k = 4");


Note: these plots can also be exported to LaTeX, in a quality 
that is suitable for print publication:
```csharp
ErrorPlot.ToGnuplot().PlotCairolatex().SaveTo("C:\\tmp\\errplt.tex");


# 5 experimental convergence plot

If the exact solution is not known, one can only estimate the convergence
behavior experimentally. 
**BoSSS** provides some utility for this, the **DGFieldComparison**
class, which has a versatile, yet complex interface.

However, there is a simple interface in the workflow management toolbox.

We can augment the current session table with experimental errors:

In [ ]:

BoSSSshell.WorkflowMgm.hConvergence.Update();

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;

We observe, that columns have been added to the session table, 
starting with a prefix **L2Error\_**

In [ ]:

Tab.GetColumnNames().Skip(46)

index,value
0,saveperiod
1,rollingSaves
2,dtMin
3,dtMax
4,ImmediatePlotPeriod
5,SuperSampling
6,savetodb
7,AlternateDbPaths[0].Item1
8,logFileDirectory
9,Paramstudy_ContinueOnError


In [ ]:
Tab = Tab.ExtractColumns(
    "DGdegree:T", "Grid:NoOfCells",
    "SolL2err", "L2Error_T");

We observe that the \emph{experimental} $L^2$ error is approximately
equal to the $L^2$ error against the exact solution, 
except for the highest resolutions. There, the error of the numerical 
solution is computed against itself, and thus the error is zero up 
to round-off errors.

If we would like to extract convergence plots from this table, we need to
exclude the rows with the finest solution using e.g. the 
**TableExtensions.ExtractRows** method.

In [ ]:

Tab.Print();

    DGdegree:T Grid:NoOfCells SolL2err               L2Error_T              
0:  4          1024           4.189135477771225E-09  0                      
1:  3          1024           3.3925701613103797E-07 0                      
2:  4          256            1.3463890974512488E-07 1.3498329517654062E-07 
3:  3          256            5.584910636792265E-06  5.598437136648202E-06  
4:  4          64             4.366082710599254E-06  4.36614321070038E-06   
5:  3          64             9.849416925455182E-05  9.850041642704084E-05  
6:  4          16             0.00014480970262714088 0.00014481001498229605 
7:  4          4              0.004890481980291057   0.004890617405529352   
8:  3          16             0.001914897579083584   0.00191490652800203    
9:  3          4              0.03377668103207887    0.03377830444812024    
10: 2          1024           2.568520840537063E-05  0                      
11: 2          256            0.00022420348456938434 0.00022048647858521555 

Rows could be extracted form a table using a selector function:
this is an expression, which is true for all rows that we want to extract;

In [ ]:

Tab = Tab.ExtractRows(
  (iRow, RowEntries) => Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != 1024);

In [ ]:
Tab.Print();

    DGdegree:T Grid:NoOfCells SolL2err               L2Error_T              
0:  4          256            1.3463890974512488E-07 1.3498329517654062E-07 
1:  3          256            5.584910636792265E-06  5.598437136648202E-06  
2:  4          64             4.366082710599254E-06  4.36614321070038E-06   
3:  3          64             9.849416925455182E-05  9.850041642704084E-05  
4:  4          16             0.00014480970262714088 0.00014481001498229605 
5:  4          4              0.004890481980291057   0.004890617405529352   
6:  3          16             0.001914897579083584   0.00191490652800203    
7:  3          4              0.03377668103207887    0.03377830444812024    
8:  2          256            0.00022420348456938434 0.00022048647858521555 
9:  2          64             0.0022755216851427103  0.002271617140769677   
10: 2          16             0.02586482501943587    0.02586029489597065    
11: 2          4              0.23642809817535057    0.23643733964892708    


# 5 Working without the session table

As an alternative to working with the session table, which is sometimes
not versatile enough, we demonstrate a way to extract data 
from the sessions in the current project directly.

Create a list in which we store a separate plot for each polynomial degree:

In [ ]:
var ExpPlotS = new List<Plot2Ddata>();
 
foreach(int pDeg in Degrees) { // loop over polynomial degrees
    // extract sessions with DG degree pDeg
    var pDegSessions = BoSSSshell.WorkflowMgm.Sessions.Where(
              // function which is true on all sessions we are interested in:
              Si => Convert.ToInt32(Si.KeysAndQueries["DGdegree:T"]) == pDeg
        ).ToArray();
 
    // now, create a plot from the selected sessions:
    // (we could also do other things)
    Plot2Ddata pDegPlot = 
        pDegSessions.ToEstimatedGridConvergenceData("T", 
             xAxis_Is_hOrDof:false, // false selects DOFs for x-axis
             normType:NormType.H1_approximate); // use the H1-Sobolev norm 
 
    // remember the freshly created plot object in a list:
    ExpPlotS.Add(pDegPlot);
}

We adjust some plot style settings:

In [ ]:
ExpPlotS[0].dataGroups[0].Format.PointType = PointTypes.Diamond;
ExpPlotS[1].dataGroups[0].Format.PointType = PointTypes.Box;
ExpPlotS[2].dataGroups[0].Format.PointType = PointTypes.LowerTriangle;
ExpPlotS[3].dataGroups[0].Format.PointType = PointTypes.Asterisk;
ExpPlotS[0].dataGroups[0].Name = "$k = 1$";
ExpPlotS[1].dataGroups[0].Name = "$k = 2$";
ExpPlotS[2].dataGroups[0].Name = "$k = 3$";
ExpPlotS[3].dataGroups[0].Name = "$k = 4$";
foreach(var p in ExpPlotS) {
    p.dataGroups[0].Format.PointSize = 0.8;
    p.dataGroups[0].Format.DashType = DashTypes.Dotted;
    p.dataGroups[0].Format.LineWidth = 2;
}

and we can merge all four plot objects into a singe one:

In [ ]:
var ExpPlot = ExpPlotS[0]; //           select 0-th object
foreach(var p in ExpPlotS.Skip(1)) { // loop over other (skip 0-th entry)
    ExpPlot = ExpPlot.Merge(p); //      merge 
}

In [ ]:
ExpPlot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $k = 1$ 
 
 
 
 
 $k = 1$ 
 
 
 
 
 
 
 
 
 
 
 $k = 2$ 
 
 
 $k = 2$ 
 
 
 
 
 
 
 
 
 
 
 $k = 3$ 
 
 
 $k = 3$ 
 
 
 
 
 
 
 
 
 
 
 $k = 4$ 
 
 
 $k = 4$

and we can also verify the slope of the error curves. 
Note that convergence order by using the $H^1$ norm is one degree lower 
compared to the $L^2$ norm..

In [ ]:

ExpPlot.Regression()

index,Key,Value
0,$k = 1$,-1.2209606105228543
1,$k = 2$,-2.1268761491492527
2,$k = 3$,-3.099454490649745
3,$k = 4$,-4.050684955077422


In [ ]:
/// BoSSScmdSilent
var regExp   = ExpPlot.Regression();
double conv1 = regExp.Single(kv => kv.Key.Contains("1")).Value;
double conv2 = regExp.Single(kv => kv.Key.Contains("2")).Value;
double conv3 = regExp.Single(kv => kv.Key.Contains("3")).Value;
double conv4 = regExp.Single(kv => kv.Key.Contains("4")).Value;
Assert.IsTrue(Math.Abs(conv1 - (-1)) < 0.7, "experimental convergence failed on k = 1");
Assert.IsTrue(Math.Abs(conv2 - (-2)) < 0.7, "experimental convergence failed on k = 2");
Assert.IsTrue(Math.Abs(conv3 - (-3)) < 0.7, "experimental convergence failed on k = 3");
Assert.IsTrue(Math.Abs(conv4 - (-4)) < 0.7, "experimental convergence failed on k = 4");


# 6 Multiplot demonstration

If we have more than one plot object, we can arrange them in an array 
to realize multi-plots:

In [ ]:
var multiplot = new Plot2Ddata[2,2];

In [ ]:
multiplot[0,0] = ExpPlotS[0];
multiplot[0,1] = ExpPlotS[1];
multiplot[1,0] = ExpPlotS[2];
multiplot[1,1] = ExpPlotS[3];

Now, we can draw an array of plots:

In [ ]:
multiplot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $k = 1$ 
 
 
 
 
 $k = 1$ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $k = 2$ 
 
 
 
 
 $k = 2$ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $k = 3$ 
 
 
 
 
 $k = 3$ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $k = 4$ 
 
 
 
 
 $k = 4$

this already looks neat, but a few 
formatting tweaks to make the multi-plot look nicer:

In [ ]:

multiplot[0,1].dataGroups[0].UseY2 = true; // label right on right column
multiplot[0,1].ShowYtics = false;
multiplot[0,1].ShowY2tics = true;
 
multiplot[1,1].dataGroups[0].UseY2 = true; // label right on right column
multiplot[1,1].ShowYtics = false;
multiplot[1,1].ShowY2tics = true;
 
multiplot[0,0].dataGroups[0].UseX2 = true; // label on top on top row
multiplot[0,0].ShowXtics = false;
multiplot[0,0].ShowX2tics = true;
 
multiplot[0,1].dataGroups[0].UseX2 = true; // label on top on top row
multiplot[0,1].ShowXtics = false;
multiplot[0,1].ShowX2tics = true;
 
// turn logarithm on for the secondary axis; 
multiplot[0,0].LogX2 = true;
multiplot[0,1].LogX2 = true;
multiplot[1,0].LogX2 = true;
multiplot[1,1].LogX2 = true;
multiplot[0,0].LogY2 = true;
multiplot[0,1].LogY2 = true;
multiplot[1,0].LogY2 = true;
multiplot[1,1].LogY2 = true;
 
// set x ranges
multiplot[0,0].X2rangeMin = 1e0;
multiplot[0,0].X2rangeMax = 1e2;
multiplot[0,1].X2rangeMin = 1e0;
multiplot[0,1].X2rangeMax = 1e2;
 
multiplot[1,0].XrangeMin = 1e0;
multiplot[1,0].XrangeMax = 1e2;
multiplot[1,1].XrangeMin = 1e0;
multiplot[1,1].XrangeMax = 1e2;
 
// set y ranges
multiplot[0,0].YrangeMin = 1e-7;
multiplot[0,0].YrangeMax = 1e0;
multiplot[1,0].YrangeMin = 1e-7;
multiplot[1,0].YrangeMax = 1e0;
 
multiplot[0,1].Y2rangeMin = 1e-7;
multiplot[0,1].Y2rangeMax = 1e0;
multiplot[1,1].Y2rangeMin = 1e-7;
multiplot[1,1].Y2rangeMax = 1e0;
 
// reduce the whitespace in between the plots:
multiplot[0,0].rmargin = 2;
multiplot[0,1].lmargin = 2;
multiplot[1,0].rmargin = 2;
multiplot[1,1].lmargin = 2;
multiplot[0,0].bmargin = 0.5;
multiplot[1,0].tmargin = 0.5;
multiplot[0,1].bmargin = 0.5;
multiplot[1,1].tmargin = 0.5;

In [ ]:
multiplot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -7 $ 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 $k = 1$ 
 
 
 
 
 $k = 1$ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -7 $ 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 $k = 2$ 
 
 
 
 
 $k = 2$ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -7 $ 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $k = 3$ 
 
 
 
 
 $k = 3$ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 -7 $ 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $k = 4$ 
 
 
 
 
 $k = 4$

# 6 Summary

This tutorial showed how to set-up a parameter study,
by looping over a set of parameters (in this case, different grids
and polynomial degrees), see sections about MeshCreation
and about Setup-And-Execution.
Finally, it only requires a simple loop to send all jobs to a 
compute resource.

Afterwards, c.f. section about ExactConvergence,
the **session table** was used to combine measurements
taken in each session (here, the $L^2$ error against the exact solution)
into a single table.
This table can either be exported to spreadsheet analysis software
or visualized internally.